## Activate GPU **(Colab only)**

When in Colab, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

In [1]:
#@title Notebook Setup

#@markdown In order to be able to run the code, we need to install the *eagerx_tutorials* package.

try:
    import eagerx_tutorials
except ImportError:
    !{"echo 'Installing eagerx-tutorials with pip.' && pip install eagerx-tutorials >> /tmp/eagerx_install.txt 2>&1"}
    
try:
    import huggingface_sb3
except ImportError:
    !{"echo 'Installing gym with pip.' && pip install gym==0.26 >> /tmp/gym.txt 2>&1"}
    !{"echo 'Installing huggingface-sb3 with pip.' && pip install git+https://github.com/huggingface/huggingface_sb3@gymnasium >> /tmp/eagerx_huggingface.txt 2>&1"}
    !{"echo 'Installing pickle for loading policies.' && pip install --upgrade --quiet cloudpickle pickle5 >> /tmp/eagerx_pickle.txt 2>&1"}

# Setup interactive notebook
# Required in interactive notebooks only.
from eagerx_tutorials import helper
helper.setup_notebook()

# Import eagerx
import eagerx
eagerx.set_log_level(eagerx.WARN)

Not running on CoLab.
Installing eagerx-gui



# EAGERx Getting Started with EAGERx

EAGERx: https://github.com/eager-dev/eagerx

Documentation: https://eagerx.readthedocs.io/en/master/

## Introduction

The objective of this tutorial is to develop a trained policy for swinging up the renowned Gym [Pendulum](https://www.gymlibrary.ml/environments/classic_control/pendulum/) and transferring this policy to a physical pendulum system. However, it should be noted that this transfer won't work seamlessly due to variations in the mass and length between the Gym pendulum and the real pendulum.

To overcome this disparity and create a policy that can effectively swing up the real pendulum system, we will employ a technique called [domain randomization](https://sites.google.com/view/domainrandomization/). By introducing randomization in the mass and length parameters of the Gym pendulum, we can train a policy that adapts to different configurations without requiring precise knowledge of the real pendulum's mass and length.

<img src="./figures/gym_pendulum.gif" width="480" /> <img src="./figures/real_pendulum.gif" width="480" />

**Figure 1:** On the left we see the Gym pendulum and on the right the real pendulum system.
The real pendulum system consists of a mass attached to a rotating disc, which dynamics are similar to those of a pendulum with a mass attached to the tip.


This tutorial covers:
- constructing a [Graph](https://eagerx.readthedocs.io/en/master/guide/api_reference/graph/graph.html) and [Environment](https://eagerx.readthedocs.io/en/master/guide/api_reference/env/index.html) with [EAGERx](https://eagerx.readthedocs.io/en/master/),
- switching between different [Engines](https://eagerx.readthedocs.io/en/master/guide/api_reference/engine/index.html),
- performing domain randomization.

In the remainder of this tutorial, we will go more into detail on these concepts.

Furthermore, you will be asked to add/modify a couple of lines of code, which are marked by

```python

# YOUR CODE HERE

# END OF YOUR CODE
```

## Pendulum Swing-up

We will create an environment for solving the classic control problem of swinging up an underactuated pendulum, i.e. the [Pendulum-v1 environment](https://www.gymlibrary.ml/environments/classic_control/pendulum/).
Our goal is to transfer the policy to the real system and swing up this pendulum to the upright position and keep it there, while minimizing the velocity of the pendulum and the input voltage.

Since the dynamics of a pendulum actuated by a DC motor are well known, we can simulate the pendulum by integrating the corresponding ordinary differential equations (ODEs):


$\mathbf{x} = \begin{bmatrix} \theta \\ \dot{\theta} \end{bmatrix} \\ \dot{\mathbf{x}} = \begin{bmatrix} \dot{\theta} \\ \frac{1}{J}(\frac{K}{R}u - mgl \sin{\theta} - b \dot{\theta} - \frac{K^2}{R}\dot{\theta})\end{bmatrix}$

with $\theta$ the angle w.r.t. upright position, $\dot{\theta}$ the angular velocity, $u$ the input voltage, $J$ the inertia, $m$ the mass, $g$ the gravitational constant, $l$ the length of the pendulum, $b$ the motor viscous friction constant, $K$ the motor constant and $R$ the electric resistance.

In [2]:
#@title Let's get started

#@markdown Next, we will download a pretrained policy in order to see what a successful policy looks like.

import sys
import stable_baselines3 as sb3
from huggingface_sb3 import load_from_hub

# Download pretrained policy from hugging face
newer_python_version = sys.version_info.major == 3 and sys.version_info.minor >= 8
custom_objects = {}
if newer_python_version:
    custom_objects = {
        "learning_rate": 0.0,
        "lr_schedule": lambda _: 0.0,
        "clip_range": lambda _: 0.0,
    }
checkpoint = load_from_hub(
    repo_id="sb3/ppo-Pendulum-v1",
    filename="ppo-Pendulum-v1.zip",
)

# Initialize model
pretrained_model = sb3.PPO.load(checkpoint, custom_objects=custom_objects, device="cpu")

/home/r2ci/.cache/pypoetry/virtualenvs/eagerx-tutorials-t4w5hBSU-py3.8/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:95: UserWarning: You loaded a model that was trained using OpenAI Gym. We strongly recommend transitioning to Gymnasium by saving that model again.
  warnings.warn(


In [3]:
#@markdown We will create a standard *Pendulum-v1* Gym environment in order to evaluate the policy.

import gym

# Initalize pendulum environment
env = gym.make("Pendulum-v1", render_mode="rgb_array")

# Evaluate policy and record video
helper.record_video(env=env, model=pretrained_model, prefix="pretrained")

# Show video
helper.show_video("pretrained-step-0-to-step-500")

/home/r2ci/.cache/pypoetry/virtualenvs/eagerx-tutorials-t4w5hBSU-py3.8/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Saving video to /home/r2ci/eagerx-dev/eagerx_tutorials/tutorials/icra/solutions/videos/pretrained-step-0-to-step-500.mp4
Moviepy - Building video /home/r2ci/eagerx-dev/eagerx_tutorials/tutorials/icra/solutions/videos/pretrained-step-0-to-step-500.mp4.
Moviepy - Writing video /home/r2ci/eagerx-dev/eagerx_tutorials/tutorials/icra/solutions/videos/pretrained-step-0-to-step-500.mp4



Moviepy - Done !
Moviepy - video ready /home/r2ci/eagerx-dev/eagerx_tutorials/tutorials/icra/solutions/videos/pretrained-step-0-to-step-500.mp4


We see that the pretrained policy is able to swing up the pendulum and stabilize it upright.
But will this policy also work on the real system?
Using EAGERx we can create environments that are engine agnostic, i.e. Gym environments that can be used with different [Engines](https://eagerx.readthedocs.io/en/master/guide/api_reference/engine/index.html?highlight=engine).
An engine could be a simulator, like PyBullet, but it could also be the real world.
Before creating the environment, we will create an [Object](https://eagerx.readthedocs.io/en/master/guide/api_reference/object/index.html?highlight=Object).
In EAGERx, an `Object` is an entity that has inputs (actuators), outputs (sensors) and states (that can be reset at the beginning of an episode).

We are going to create one object (the pendulum).
For this first tutorial, we don't want to go into details too much and start with existing objects.
Note that we import the pendulum.
While this might look like an unused import, it is not.
During the import, the pendulum object is registered and we can therefore make it based on its ID, i.e. *Pendulum*.

Before making the object, we will first obtain some info on the *Pendulum*, such that we know with what arguments we should make it.


In [4]:
from eagerx_tutorials.pendulum.objects import Pendulum
import eagerx_tutorials.pendulum.gym_implementation  # NOOP to register Gym implementation of the pendulum.

Pendulum.info()

   entity_type: `Pendulum`
   module: `eagerx_tutorials.pendulum.objects`
   file: `/home/r2ci/eagerx-dev/eagerx_tutorials/eagerx_tutorials/pendulum/objects.py`

Supported engines:
 - eagerx_ode.engine/OdeEngine
 - eagerx.engines.openai_gym.engine/GymEngine

Make this spec with:
   spec = Pendulum.make(name: str, actuators: List[str] = None, sensors: List[str] = None, states: List[str] = None, rate: float = 30.0, render_shape: List[int] = None, render_fn: str = None)

class Pendulum:
   make(name: str, actuators: List[str] = None, sensors: List[str] = None, states: List[str] = None, rate: float = 30.0, render_shape: List[int] = None, render_fn: str = None):
      sensors:
       - theta: Space(-999.0, 999.0, (), float32)
       - theta_dot: Space(-999.0, 999.0, (), float32)
       - image: Space(uint8)
       - u_applied: Space([-2.], [2.], (1,), float32)
      actuators:
       - u: Space([-2.], [2.], (1,), float32)
      engine_states:
       - model_state: Space([-3.14 -9.  ], [3.14

We see that the `Pendulum.info()` provides us information on the *Pendulum* object.
It has four sensors (`theta`, `theta_dot`, `image`, `u_applied`), one actuator (`u`) and a number of states.
Here `theta`, `theta_dot` and `u` correspond to $\theta$, $\dot{\theta}$ and $u$, respectively.

In [5]:
#@markdown We can make the parameter specification of the `Pendulum` object with the `Pendulum.make` method and can then add it to a [Graph](https://eagerx.readthedocs.io/en/master/guide/api_reference/graph/graph.html).

# Define rate (Hz)
rate = 20.0

# Initialize empty graph
graph = eagerx.Graph.create()

# Select sensors, actuators and states of Pendulum
sensors = ["theta", "theta_dot", "image"]
actuators = ["u"]
states = ["model_state", "mass", "length", "max_speed"]

# Make pendulum
pendulum = Pendulum.make("pendulum", rate=rate, actuators=actuators, sensors=sensors, states=states, render_fn="disc_pendulum_render_fn")

# Decompose angle [cos(theta), sin(theta)]
from eagerx_tutorials.pendulum.processor import DecomposedAngle
pendulum.sensors.theta.processor = DecomposedAngle.make()
pendulum.sensors.theta.space.low = -1
pendulum.sensors.theta.space.high = 1
pendulum.sensors.theta.space.shape = [2]

# Add pendulum to the graph
graph.add(pendulum)

# Connect the pendulum to an action and observations
graph.connect(action="voltage", target=pendulum.actuators.u)
graph.connect(source=pendulum.sensors.theta, observation="angle")
graph.connect(source=pendulum.sensors.theta_dot, observation="angular_velocity")

# Render image
graph.render(source=pendulum.sensors.image, rate=rate)

The graph describes the interconnection of nodes and objects.
In this way, the creation of an environment becomes modular.
This allows users to create an implementation for nodes and objects once, and easily create new environments by reusing these implementations.
Also, this allows to construct complex environments using nodes and objects as basic building blocks.

After adding the pendulum to the graph, we will connect the actuator `u` to a new action called `voltage`.
We will connect the sensors `theta` and `theta_dot` to the observations `angle` and `angular_velocity`, respectively.
In this way, the agent will be able to send actions to control $u$ of the pendulum and observe $\theta$ and $\dot{\theta}$.

Finally, we also render the `image` sensor in order to visualize the pendulum.

It is also possible to inspect the graph using the eagerx-gui package.

It can be installed as follows:
```bash
pip3 install eagerx-gui
```
Colab has limited support for interactive applications, so we cannot open the GUI here.
But if we were to run
```python
graph.gui()
```
The output would be as follows:

<img src="../figures/tutorial_1_gui.svg" width=720 />

Here we see that the actions of the agent are outputs of *env/actions* and that the observations of the agent are inputs of *env/observations*.
Also, we render output by connecting to *env/render*.
Note that *env/actions*, *env/observations* and *env/render* represent connections of the `Graph` to the environment.
They are split up in the GUI as nodes for visualization purposes.

Next, we will create the [Environment](https://eagerx.readthedocs.io/en/master/guide/api_reference/env/index.html).
Environment creation in EAGERx follows the same API as Gym, i.e. we have to define a [step()](https://eagerx.readthedocs.io/en/master/guide/api_reference/env/index.html#eagerx.core.env.BaseEnv.step) and [reset()](https://eagerx.readthedocs.io/en/master/guide/api_reference/env/index.html#eagerx.core.env.BaseEnv.reset) method.

In [6]:
from typing import Dict
import numpy as np


class PendulumEnv(eagerx.BaseEnv):
    def __init__(self, name: str, rate: float, graph: eagerx.Graph, engine: eagerx.specs.EngineSpec, eval: bool, render_mode: str = "rgb_array"):
        """Initializes an environment with EAGERx dynamics.

        :param name: The name of the environment. Everything related to this environment
                     (parameters, topics, nodes, etc...) will be registered under namespace: "/[name]".
        :param rate: The rate (Hz) at which the environment will run.
        :param graph: The graph consisting of nodes and objects that describe the environment's dynamics.
        :param engine: The physics engine that will govern the environment's dynamics.
        :param eval: If True we will create an evaluation environment, i.e. not performing domain randomization.
        :param render_mode: Defines the render mode (e.g. "human", "rgb_array").
        """
        # Make the backend specification
        from eagerx.backends.single_process import SingleProcess
        backend = SingleProcess.make()
        
        self.eval = eval
        
        # Maximum episode length
        self.max_steps = 270 if eval else 100
        
        # Step counter
        self.steps = None
        super().__init__(name, rate, graph, engine, backend, force_start=True, render_mode=render_mode)
    
    def step(self, action: Dict):
        """A method that runs one timestep of the environment's dynamics.

        :params action: A dictionary of actions provided by the agent.
        :returns: A tuple (observation, reward, terminated, truncated, info).

              - observation: Observations of the current timestep that comply with
                             the :func:`~eagerx.core.env.BaseEnv.observation_space`.

              - reward: amount of reward returned after previous action

              - terminated: whether the episode has ended due to a terminal state, in which case further step() calls will
                            return undefined results

              - truncated: whether the episode has ended due to a time limit, in which case further step() calls will
                           return undefined results

              - info: contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)
        """
        # Take step
        observation = self._step(action)
        self.steps += 1
        
        # Extract observations
        cos_th, sin_th = observation["angle"][0]
        thdot = observation["angular_velocity"][0]
        u = action["voltage"][0]

        # Calculate reward
        # We want to penalize the angle error, angular velocity and applied voltage
        th = np.arctan2(sin_th, cos_th)
        cost = th**2 + 0.1 * (thdot / (1 + 10 * abs(th))) ** 2 + 0.01 * u ** 2

        # Determine termination conditions
        terminated = False
        truncated = self.steps > self.max_steps
        info = {}
        
        # Render
        if self.render_mode == "human":
            self.render()
        return observation, -cost, terminated, truncated, info
    
    def reset(self, seed=None, options=None):
        """Resets the environment to an initial state and returns an initial observation.

        :returns: The initial observation.
        """
        # Determine reset states
        states = self.state_space.sample()
        
        if self.eval:
            theta = 3.14 * np.random.uniform(low=0.75, high=1.0) * [-1,1][np.random.randint(2)]
            states["pendulum/model_state"][:] = [theta, 0.0]
        else:
            # YOUR CODE HERE
            # TODO: 
            # During training we want to vary the length and mass of the pendulum.
            # This will improve the robustness against model inaccuracies.
            # Randomly sample values for the mass and length of the pendulum.
            # Try to estimate the mass and length of the real pendulum system in Figure 1.
            # You can adjust the low and the high in the lines below to define the distributions for sampling.
            # Hint: the Gym pendulum is a rod, while the real pendulum is not.
            # They have different moments of inertia, therefore overestimating the length will help.

            # key = "[object_name]/[state_name]"
            # value should be of type np.ndarray
            
            # Sample mass (kg)
            states["pendulum/mass"] = np.random.uniform(low=0.03, high=0.05, size=()).astype("float32")            # Sample mass (kg)
            # Sample length (m)
            states["pendulum/length"] = np.random.uniform(low=0.10, high=0.14, size=()).astype("float32")            # Sample length (m)
            
            # END OF YOUR CODE
            
        # Perform reset
        observation = self._reset(states)
        info = {}

        # Reset step counter
        self.steps = 0
        
        # Render
        if self.render_mode == "human":
            self.render()
        return observation, info

In [7]:
#@markdown Next, we will create the Engines corresponding to the simulators we will use.

# Import the two supported engines
from eagerx_ode.engine import OdeEngine
from eagerx.engines.openai_gym.engine import GymEngine

# Initialize engines
gym_engine = GymEngine.make(rate=rate)
ode_engine = OdeEngine.make(rate=rate)

We will make use of the *GymEngine* to simulate the Gym pendulum and the *OdeEngine* to simulate the disc pendulum based on the ODE at the top of the page with identified parameters from the real system.
The *GymEngine* will be used for training and here we perform the domain randomization.
The *OdeEngine* will be used for evaluation in order to validate whether the resulting policy will work on the real system.
Switching between simulators or to the real world is only a matter of switching the engine in EAGERx.

In [8]:
#@title Now we are ready to make the environments! 

#@markdown We will create one with the `gym_engine` for training and one with the `ode_engine` for evaluation.

from eagerx.wrappers import Flatten


# Initialize environments
train_env = PendulumEnv(name="train", rate=rate, graph=graph, engine=gym_engine, eval=False, render_mode="human")
eval_env = PendulumEnv(name="eval", rate=rate, graph=graph, engine=ode_engine, eval=True)


# Stable Baselines3 expects flattened actions & observations
# Convert observation and action space from Dict() to Box()
train_env = Flatten(train_env)
eval_env = Flatten(eval_env)

[WARN]: Backend 'SINGLE_PROCESS' does not support multiprocessing, so all nodes are launched in the ENVIRONMENT process.


In [9]:
#@markdown Let's first check if the pretrained policy we downloaded at the beginning transfers to the simulated disc pendulum...

helper.evaluate(pretrained_model, eval_env, episode_length=270, video_rate=rate, video_prefix="pretrained_disc")

Start evaluation episode 0 of 3


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 270/270 [00:01<00:00, 238.04it/s]


Start video writer
Showing episode 0 with episodic reward: -2012.4147425682957


Start evaluation episode 1 of 3


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 270/270 [00:01<00:00, 238.10it/s]


Start video writer
Showing episode 1 with episodic reward: -1934.3245827300425


Start evaluation episode 2 of 3


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 270/270 [00:01<00:00, 238.13it/s]


Start video writer
Showing episode 2 with episodic reward: -2013.3549464778412


Finished evaluation with mean episodic reward: -1986.6980905920598


We see that the pretrained policy fails...
This was also to be expected, since the mass and length of the Gym pendulum are 1 kg and 1 m, respectively.
Therefore, we will train again a policy on the Gym pendulum, but we will now use different values for the mass and the length of the pendulum.
There is only one problem: you don't know the exact mass and length of the real pendulum system.
You can still train a successful policy however, by performing [domain randomization](https://sites.google.com/view/domainrandomization/).
By varying over different values of $m$ and $l$, you can train a policy that is robust against model inaccuracies.
In order to do this, you have to modify a few lines of code in the `reset` method of the `PendulumEnv` class.

In [10]:
#@title If you have added your code, you can train a policy as follows (this will take a couple of minutes in Colab).

#@markdown **NOTE: If you want to rerun code, we advice you to restart and run all code (in Colab there is the option Restart and run all under Runtime).**

# Initialize learner
model = sb3.SAC("MlpPolicy", train_env, verbose=1, learning_rate=7e-4)

# Train for 40 episodes
model.learn(total_timesteps=int(4000))
train_env.close()

# Save model
model.save("pendulum")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -457     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 72       |
|    time_elapsed    | 5        |
|    total_timesteps | 404      |
| train/             |          |
|    actor_loss      | 7.23     |
|    critic_loss     | 0.242    |
|    ent_coef        | 0.809    |
|    ent_coef_loss   | -0.351   |
|    learning_rate   | 0.0007   |
|    n_updates       | 303      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -393     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 76       |
|    time_elapsed    | 10       |
|    total_timesteps | 808      |
| train/             

In [11]:
#@markdown Next, you can evaluate your policy again on the simulated disc pendulum.

# Create evaluation environment
eval_env = PendulumEnv(name="disc", rate=rate, graph=graph, engine=ode_engine, eval=True)
eval_env = Flatten(eval_env)

helper.evaluate(model, eval_env, episode_length=270, video_rate=rate, video_prefix="trained_disc")

Start evaluation episode 0 of 3


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 270/270 [00:01<00:00, 205.48it/s]


Start video writer
Showing episode 0 with episodic reward: -2317.276605660357


Start evaluation episode 1 of 3


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 270/270 [00:01<00:00, 210.04it/s]


Start video writer
Showing episode 1 with episodic reward: -2342.1262181304196


Start evaluation episode 2 of 3


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 270/270 [00:01<00:00, 211.47it/s]


Start video writer
Showing episode 2 with episodic reward: -2320.1347653984917


Finished evaluation with mean episodic reward: -2326.5125297297564


And.. were you able to swing up the disc pendulum and stabilize it upright successfully?
Note that the mean episodic reward is printed.
A successful policy should result in a mean episodic reward of at least -200.